In [1]:
import torch 
from Focal_loss import focal_loss
import torch.nn as nn
from dataset import YinYingCNNDataset
from CNN import CNNArea
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim

In [14]:
# 自定义损失函数，仅对 mask 区域进行计算
class Loss(nn.Module):
    def __init__(self):
        super(Loss, self).__init__()

    def forward(self, output, label):
        squared_loss = (torch.abs(output - label))
        loss = squared_loss.sum()
        return loss

In [16]:
# 定义设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 定义模型
model = CNNArea().to(device)

# 定义优化器
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_list = []
num_epochs = 1
loss_fn = Loss()

root_dir = './data/'
dataset = YinYingCNNDataset(root_dir)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (feature,label,mask) in enumerate(dataloader):
        feature = feature.to(device)
        pred = model(feature)
        label_sum = torch.sum(label,dim=(1,2,3)).unsqueeze(1).to(device)
        loss = loss_fn(pred,label_sum) 
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    epoch_loss = running_loss / len(dataloader.dataset)
    loss_list.append(epoch_loss)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}') 

tensor(1674.7838, device='cuda:0', grad_fn=<SumBackward0>)
